In [1]:
!pip install langchain
!pip install langchain_community
!pip install PyPDFLoader
!pip install pypdf
!pip install chromadb
!pip install -U langchain-chroma
!pip install langchain-huggingface
!pip install groq
!pip install langchain-groq
!pip install sentence-transformers==2.4.0


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Laoding the data(Constitution.pdf)

In [2]:
from langchain_community.document_loaders import PyPDFLoader

def load_files(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents 

In [3]:
data = load_files("Constitution.pdf")

# Splitting the data into chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [5]:
def split_text(data_for_splitting):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)
    text_chunks = text_splitter.split_documents(data_for_splitting)
    return text_chunks

In [6]:
text_chunks = split_text(data)
print("Length of chunks is :", len(text_chunks))

Length of chunks is : 802


# Embedding

In [7]:
# !pip install -U transformers sentence-transformers


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
embedded_text = embeddings.embed_documents(d.page_content for d in text_chunks)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length of the query", len(query_result))

Length 384


In [12]:
import os
from dotenv import load_dotenv
load_dotenv('.env')

True

# Vector Database

In [13]:
import chromadb

client = chromadb.PersistentClient('Vector_db')
collections = client.get_or_create_collection(name = 'pdf_collection')

In [14]:
collections.add(
    documents = [d.page_content for d in text_chunks],
    metadatas = [d.metadata for d in text_chunks],
    embeddings = embedded_text,
    ids = [str(i) for i in range(len(text_chunks))]
)    

In [15]:
from langchain_chroma import Chroma

vector_db = Chroma(
    collection_name = 'pdf_collection',
    persist_directory = 'Vector_db'
)

retriever = vector_db.as_retriever(search_type = 'similarity', search_kwargs={'k':3})


# Setting up the LLM

In [20]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = os.getenv('GROQ_API_KEY'),
    model_name="llama-3.3-70b-versatile"
)

# Creating the RetrievalQA chain

In [21]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    return_source_documents=True
)

# # Customizing the prompt

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a highly respected Senior Advocate of Pakistan and expert in Pakistan law. "
    "Use the following pieces of retrieved legal context, including the Pakistan Constitution, Acts, and case law, "
    "to answer the question as it applies to real-life Pakistan scenarios. If you don't know the answer, clearly say you don't know. "
    "Keep your response concise, professional, and informative — a maximum of three sentences. "
    "Where relevant, cite sections of the Constitution or law. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [18]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What does constitution says about freedom of expression"})
print(response["answer"])

According to Article 19 of the Constitution of Pakistan, every citizen has the right to freedom of speech and expression, and there shall be freedom of the press, subject to any reasonable restrictions imposed by law in the interest of the glory of Islam, integrity, security, or defence of Pakistan, among other considerations. This freedom is not absolute and can be restricted by law if necessary. The Constitution also provides for the right to information under Article 19A, which is closely related to the freedom of expression.


In [19]:
# pip uninstall -y transformers tokenizers sentence-transformers
